In [1]:
import numpy as np
import pandas as pd
import pandahouse as ph

# 1

In [2]:
connection_from = {
    'host': 'https://clickhouse.lab.karpov.courses/',
    'database':'simulator_20220320',
    'user':'student', 
    'password':'dpo_python_2020'
}

In [3]:
query_actions = '''
select
toDate(time) as event_date,
user_id,
gender,
age,
os,
countIf(action='view') as views,
countIf(action='like') as likes
from simulator_20220320.feed_actions
where toStartOfDay(time) = yesterday()
group by event_date, user_id, gender, age, os
'''
actions = ph.read_clickhouse(query_actions, connection=connection_from)

In [4]:
actions

,event_date,user_id,gender,age,os,views,likes
0,2022-04-23,71712,1,30,Android,15,0
1,2022-04-23,12652,0,22,iOS,46,8
2,2022-04-23,116635,0,29,Android,26,9
3,2022-04-23,124036,0,14,Android,33,11
4,2022-04-23,54152,1,18,Android,9,3
...,...,...,...,...,...,...,...
19736,2022-04-23,4714,0,41,iOS,27,2
19737,2022-04-23,124054,1,22,iOS,18,2
19738,2022-04-23,60105,1,28,iOS,26,0
19739,2022-04-23,115743,1,45,Android,27,5


In [5]:
query_messages = '''
with
t1 as (
    select
    toDate(time) as event_date,
    user_id,
    count(user_id) as sent,
    uniqExact(reciever_id) as recievers
    from simulator_20220320.message_actions
    where toStartOfDay(time) = yesterday()
    group by event_date, user_id
),
t2 as (    
    select
    toDate(time) as event_date,
    reciever_id,
    count(reciever_id) as recieved,
    uniqExact(user_id) as senders
    from simulator_20220320.message_actions
    where toStartOfDay(time) = yesterday()
    group by event_date, reciever_id
),
interim as (
    select
    if(t1.event_date = '1970-01-01', t2.event_date, t1.event_date) as event_date,
    if(t1.user_id = 0, t2.reciever_id, t1.user_id) as user_id,
    t1.sent as messages_sent,
    t1.recievers as users_sent,
    t2.recieved as messages_received,
    t2.senders as users_received
    from t1 full join t2 on t1.user_id = t2.reciever_id
),
attr as (
select distinct user_id, gender, age, os
from simulator_20220320.message_actions
)
select
    l.event_date,
    l.user_id,
    r.gender,
    r.age,
    r.os,
    l.messages_sent,
    l.users_sent,
    l.messages_received,
    l.users_received
from interim as l
left join attr as r
on l.user_id = r.user_id
'''
messages = ph.read_clickhouse(query_messages, connection=connection_from)

In [6]:
messages

,event_date,user_id,gender,age,os,messages_sent,users_sent,messages_received,users_received
0,2022-04-23,123845,0,35,iOS,10,2,0,0
1,2022-04-23,116371,0,21,iOS,7,7,0,0
2,2022-04-23,5213,0,31,iOS,13,11,17,16
3,2022-04-23,15858,0,22,iOS,4,4,7,7
4,2022-04-23,3275,1,18,iOS,8,8,2,2
...,...,...,...,...,...,...,...,...,...
1881,2022-04-23,123002,1,15,Android,0,0,1,1
1882,2022-04-23,19342,1,26,iOS,0,0,3,3
1883,2022-04-23,5751,1,38,iOS,0,0,2,2
1884,2022-04-23,9070,0,24,Android,0,0,4,3


In [7]:
messages.user_id.nunique()

1886

# 2

In [8]:
df = pd.merge(actions, messages, on=['event_date', 'user_id', 'gender', 'age', 'os'], how='outer')

In [9]:
def age_group(age):
    if age < 18:
        return '0-17'
    elif age < 25:
        return '18-24'
    elif age < 41:
        return '25-40'
    elif age < 56:
        return '31-55'
    else:
        return '56+'

In [10]:
def gender_str(x):
    if x == 1:
        return 'male'
    else:
        return 'female'

In [11]:
df['age'] = df['age'].apply(age_group)
df['gender'] = df['gender'].apply(gender_str)
df.fillna(0, inplace=True)

In [12]:
df

,event_date,user_id,gender,age,os,views,likes,messages_sent,users_sent,messages_received,users_received
0,2022-04-23,71712,male,25-40,Android,15.0,0.0,0.0,0.0,0.0,0.0
1,2022-04-23,12652,female,18-24,iOS,46.0,8.0,0.0,0.0,0.0,0.0
2,2022-04-23,116635,female,25-40,Android,26.0,9.0,0.0,0.0,0.0,0.0
3,2022-04-23,124036,female,0-17,Android,33.0,11.0,0.0,0.0,0.0,0.0
4,2022-04-23,54152,male,18-24,Android,9.0,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
21391,2022-04-23,123002,male,0-17,Android,0.0,0.0,0.0,0.0,1.0,1.0
21392,2022-04-23,19342,male,25-40,iOS,0.0,0.0,0.0,0.0,3.0,3.0
21393,2022-04-23,5751,male,25-40,iOS,0.0,0.0,0.0,0.0,2.0,2.0
21394,2022-04-23,9070,female,18-24,Android,0.0,0.0,0.0,0.0,4.0,3.0


# 3.A

In [13]:
def calc_aggs_by_metric(tbl, metric):
    tbl['metric'] = metric
    aggs = tbl.groupby(by=['event_date', 'metric', metric])[['views', 'likes', 'messages_sent', 'users_sent', 'messages_received', 'users_received']].sum().reset_index()
    aggs.rename(columns={metric:'metric_value'},inplace=True)
    return aggs

In [14]:
gender_aggs = calc_aggs_by_metric(tbl=df, metric='gender')
gender_aggs

,event_date,metric,metric_value,views,likes,messages_sent,users_sent,messages_received,users_received
0,2022-04-23,gender,female,211235.0,41421.0,5543.0,4285.0,5532.0,4138.0
1,2022-04-23,gender,male,259892.0,51028.0,7357.0,5670.0,7368.0,5817.0


In [15]:
age_aggs = calc_aggs_by_metric(tbl=df, metric='age')
age_aggs

,event_date,metric,metric_value,views,likes,messages_sent,users_sent,messages_received,users_received
0,2022-04-23,age,0-17,62740.0,12188.0,1874.0,1473.0,1482.0,1179.0
1,2022-04-23,age,18-24,182894.0,36044.0,4894.0,4744.0,5565.0,4766.0
2,2022-04-23,age,25-40,168613.0,33128.0,4605.0,3236.0,5183.0,3776.0
3,2022-04-23,age,31-55,49499.0,9707.0,1388.0,462.0,448.0,172.0
4,2022-04-23,age,56+,7381.0,1382.0,139.0,40.0,222.0,62.0


In [16]:
os_aggs = calc_aggs_by_metric(tbl=df, metric='os')
os_aggs

,event_date,metric,metric_value,views,likes,messages_sent,users_sent,messages_received,users_received
0,2022-04-23,os,Android,307058.0,60101.0,8312.0,6467.0,8530.0,6564.0
1,2022-04-23,os,iOS,164069.0,32348.0,4588.0,3488.0,4370.0,3391.0


In [17]:
pd.concat([gender_aggs, age_aggs, os_aggs]).reset_index(drop=True)

,event_date,metric,metric_value,views,likes,messages_sent,users_sent,messages_received,users_received
0,2022-04-23,gender,female,211235.0,41421.0,5543.0,4285.0,5532.0,4138.0
1,2022-04-23,gender,male,259892.0,51028.0,7357.0,5670.0,7368.0,5817.0
2,2022-04-23,age,0-17,62740.0,12188.0,1874.0,1473.0,1482.0,1179.0
3,2022-04-23,age,18-24,182894.0,36044.0,4894.0,4744.0,5565.0,4766.0
4,2022-04-23,age,25-40,168613.0,33128.0,4605.0,3236.0,5183.0,3776.0
5,2022-04-23,age,31-55,49499.0,9707.0,1388.0,462.0,448.0,172.0
6,2022-04-23,age,56+,7381.0,1382.0,139.0,40.0,222.0,62.0
7,2022-04-23,os,Android,307058.0,60101.0,8312.0,6467.0,8530.0,6564.0
8,2022-04-23,os,iOS,164069.0,32348.0,4588.0,3488.0,4370.0,3391.0


# 3.B

In [18]:
df.groupby(['event_date', 'gender', 'age', 'os'])[['views', 'likes', 'messages_sent', 'users_sent', 'messages_received', 'users_received']].sum().reset_index()

,event_date,gender,age,os,views,likes,messages_sent,users_sent,messages_received,users_received
0,2022-04-23,female,0-17,Android,18324.0,3477.0,539.0,414.0,341.0,315.0
1,2022-04-23,female,0-17,iOS,10373.0,2087.0,341.0,264.0,155.0,145.0
2,2022-04-23,female,18-24,Android,53867.0,10589.0,1373.0,1344.0,1692.0,1401.0
3,2022-04-23,female,18-24,iOS,27632.0,5405.0,772.0,740.0,927.0,709.0
4,2022-04-23,female,25-40,Android,49935.0,9851.0,1347.0,926.0,1509.0,1054.0
5,2022-04-23,female,25-40,iOS,26120.0,5033.0,579.0,409.0,389.0,367.0
6,2022-04-23,female,31-55,Android,14503.0,2931.0,326.0,105.0,15.0,13.0
7,2022-04-23,female,31-55,iOS,7117.0,1427.0,195.0,63.0,282.0,72.0
8,2022-04-23,female,56+,Android,2026.0,367.0,23.0,8.0,0.0,0.0
9,2022-04-23,female,56+,iOS,1338.0,254.0,48.0,12.0,222.0,62.0


# creating table

In [19]:
connection_to = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': '656e2b0c9c',
    'user': 'student-rw',
    'database': 'test'
}

In [20]:
create_table_v1 = """
create table if not exists test.avs_v1
(
    event_date Date,
    gender String,
    age String,
    os String,
    views Int32,
    likes Int32,
    messages_sent Int32,
    users_sent Int32,
    messages_received Int32,
    users_received Int32
)
engine = MergeTree
order by event_date
"""

In [21]:
create_table_v2 = """
create table if not exists test.avs_v2
(
    event_date Date,
    metric String,
    metric_value String,
    views Int32,
    likes Int32,
    messages_sent Int32,
    users_sent Int32,
    messages_received Int32,
    users_received Int32
)
engine = MergeTree
order by event_date
"""

In [22]:
if ph.read_clickhouse(query='exists test.avs_v1', connection=connection_to).iloc[0, 0] == 0:
    ph.execute(connection=connection_to, query=create_table_v1)

In [23]:
ph.read_clickhouse(query='exists test.avs_v1', connection=connection_to).iloc[0, 0] == 0

False